In [ ]:
from enum import Enum
from typing import Tuple
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
import tkinter.filedialog as filedialog
import tkinter.messagebox as messagebox
import os
import csv
import time
import smtplib as smtp
import copy

# ファイル選択ダイアログで選択するファイルの種類を定義するクラス
class FileType(Enum):
  CSV = ('CSVファイルを選択', 'csv files', '*.csv')
  TXT = ('テキストファイルを選択', 'text files', '*.txt')

# ファイル選択ダイアログで指定したCSV, TXTファイルのデータを取得する
def get_file_path(filetype: FileType) -> str:
  file_path: str = ''

  # ファイル選択ダイアログの表示, 選択されたファイルパスの取得
  select_file_path = filedialog.askopenfilename(
    title = filetype.value[0],
    filetypes = [(filetype.value[1], filetype.value[2])], 
    initialdir = os.path.expanduser('~/Desktop')
    )

  # 指定された拡張子のファイルを選択していればそのファイルパスを代入
  if (filetype == FileType.CSV and select_file_path.endswith('.csv')) or (filetype == FileType.TXT and select_file_path.endswith('.txt')):
    file_path = select_file_path

  return file_path

# サーバー情報を定義するクラス
class Email:
  # CSVファイルから取得
  server_address: str = ''
  server_port: int = -1
  email_address: str = ''
  password: str = ''
  from_email_address: str = ''
  company_names: dict = {}
  customer_names: dict = {}
  to_email_addresses: dict = {}

  # TXTファイルから取得
  subject: str = ''
  attachment_file_paths: list = []
  content: str = ''

# 空白(半角/全角)を削除した文字列を返却する
def trim_space(string: str) -> str:
  return string.replace(' ', '').replace('　', '')

# CSVデータをEmailオブジェクトに追加し、送信件数を取得する
# CSVファイルは以下の内容で記述する
# server_address, port, email_address, password
# from_email_address
# company_name, customer_name, to_email_addresses1(複数ある場合は半角カンマ区切り)
# company_name, customer_name, to_email_addresses2(同上)
# company_name, customer_name, to_email_addresses3(同上)
# ...
def read_csv_file(email: Email, csv_file_path: str):
  with open(csv_file_path) as csv_file:
    reader = csv.reader(csv_file)
    
    server_info_list = trim_space(next(reader)).split(',')

    # 1行目 (SMTPサーバ情報)
    email.server_address = server_info_list[0]
    email.server_port = server_info_list[1]
    email.email_address = server_info_list[2]
    email.password = server_info_list[3]

    # 2行目 (送信元アドレス)
    email.from_email_address = next(reader)

    # 3行目以降 (会社名, 顧客名, 送信先アドレス...)
    # 会社名・顧客名はそれぞれ1社/1人しか指定できない
    to_count: int = 0
    for row in reader:
      to_count += 1
      to_info_list = trim_space(next(reader)).split(',')

      if len(to_info_list) >= 3:
        email.company_names[to_count] = to_info_list.pop(0)
        email.customer_names[to_count] = to_info_list.pop(0)
        email.to_email_addresses.setdefault(to_count, []).extend(to_info_list)

# TXTデータをEmailオブジェクトに追加
# TXTファイルは以下の内容で記述する(.txtファイルと同じディレクトリに添付ファイルを格納しておくこと)
# subject
# attachment_file_names(複数ある場合は半角スラッシュ区切り, 拡張子名も記載)
# message_contents
def read_text_file(email: Email, txt_file_path: str):
  with open(txt_file_path) as text_file:
    # 件名
    email.subject = text_file.readline()
    
    # 添付ファイル
    txt_file_dir = os.path.dirname(txt_file_path)
    attachment_file_paths: list = list(map(lambda file_path: os.path.join(txt_file_dir, file_path), trim_space(text_file.readline()).split('/')))
    for attachment_file_path in attachment_file_paths:
      if os.path.exists(attachment_file_path):
        email.attachment_file_paths.append(attachment_file_path)
    
    # メッセージ内容
    email.content = text_file.readlines()

# CSV, TXTデータを分割してEmailオブジェクトを返す
def create_email(csv_file_path: str, txt_file_path: str) -> Email:
  # CSV, TXTファイル名が取得できない場合はNoneを早期Return
  if len(csv_file_path) == 0 or len(txt_file_path) == 0:
    return None

  email = Email()

  # CSVデータの追加
  read_csv_file(email, csv_file_path)

  # TXTデータの追加
  read_text_file(email, txt_file_path)
  
  return email

# 会社名と顧客名のプレースホルダを置換する
def replace_placeholder(message: str, replace_dict: dict) -> str:
  for key in replace_dict.keys():
    message = message.replace(key, replace_dict[key])
  
  return message

# 例外に応じたエラーメッセージを返却する
def get_error_message(exception) -> str:
  if issubclass(exception, smtp.SMTPServerDisconnected):
    return 'サーバとの接続が切断されました。エラーメッセージ: ' + exception
  elif issubclass(exception, smtp.SMTPHeloError):
    return 'サーバが HELO コマンドに応答しませんでした。サーバアドレス または ポート番号 を確認してください。エラーメッセージ: ' + exception
  elif issubclass(exception, smtp.SMTPResponseException):
    return 'SMTPエラーが発生しました。SMTPエラーコード: ' + exception.smtp_code + ', SMTPエラーメッセージ: ' + exception.smtp_error
  elif issubclass(exception, smtp.SMTPSenderRefused):
    return '送信元メールアドレスを確認してください。エラーメッセージ: ' + exception
  elif issubclass(exception, smtp.SMTPRecipientsRefused):
    return '送信先メールアドレスを確認してください。エラーメッセージ: ' + exception
  elif issubclass(exception, smtp.SMTPDataError):
    return 'メッセージ内容に問題があります。エラーメッセージ: ' + exception
  elif issubclass(exception, smtp.SMTPConnectError):
    return 'サーバへの接続時にエラーが発生しました。エラーメッセージ: ' + exception
  elif issubclass(exception, smtp.SMTPHeloError):
    return 'サーバが HELO コマンドに応答しませんでした。サーバアドレス または ポート番号 を確認してください。エラーメッセージ: ' + exception
  elif issubclass(exception, smtp.SMTPNotSupportedError):
    return 'サーバが AUTH コマンドに対応していません(SMTP認証に非対応)。エラーメッセージ: ' + exception
  elif issubclass(exception, smtp.SMTPAuthenticationError):
    return 'ユーザ名(メールアドレス) または パスワードが違います。エラーメッセージ: ' + exception
  else:
    return '何らかのエラーが発生しました。エラーメッセージを確認してください。エラーメッセージ: ' + exception

# sec[秒]実行待機後にメールを逐次送信する
def send_message_with_sleep(sec: int, email: Email) -> str:
  try:
    # ベースとなるMIMEオブジェクトの生成
    base_mime = MIMEMultipart()
    # 件名
    base_mime['Subject'] = email.subject
    # 送信元アドレス
    base_mime['From'] = email.from_email_address
    # ファイル添付
    for attachment_file_path in email.attachment_file_paths:
      if os.path.exists(attachment_file_path):
        with open(attachment_file_path, 'rb') as attachment_file:
          attachment = MIMEApplication(attachment_file.read())
      else:
        raise Exception('ファイルが見つかりませんでした。ファイルパス: ' + attachment_file_path)

      attachment.add_header('Content-Disposition', 'attachment', filename=os.path.basename(attachment_file_path))
      base_mime.attach(attachment)
    
    # SMTPサーバのオブジェクト生成
    server = smtp.SMTP(email.server_address, email.server_port, timeout=10)

    if server.has_extn('STARTTLS'):
      # SSL/TLS通信の開始
      server.starttls()
    
    # メールサーバへのログイン
    server.login(email.email_address, email.password)

    # 個別設定
    replace_dict: dict = {}
    for key in email.company_names.keys:
      # 生成したベースMIMEオブジェクトをディープコピー
      mime = copy.deepcopy(base_mime)
      # 送信先アドレス
      mime['To'] = ','.join(email.to_email_addresses[key])
      # 会社名
      replace_dict['{COMPANY_NAME}'] = email.company_names[key]
      # 顧客名
      replace_dict['{CUSTOMER_NAME}'] = email.customer_names[key]
      # メッセージ内容
      mime.attach(MIMEText(replace_placeholder(email.content, replace_dict)))

      # sec[秒]の送信待機(初回は送信待機しない)
      if not key == 1:
        time.sleep(sec)

      # メールの送信
      server.send_message(mime)
    
    # サーバからの切断
    server.quit()

    return ''

  except Exception as e:
    return get_error_message(e)

# メール送信処理
def main():
  # ファイル選択ダイアログを開く
  email: Email = create_email(get_file_path(FileType.CSV), get_file_path(FileType.TXT))

  # メールの配信
  result: str = send_message_with_sleep(5, email)

  # 処理結果をダイアログに出力
  dialog: Tuple[str, str] = ('', '')
  if len(result) == 0:
    dialog[0] = '送信成功'
    dialog[1] = len(email.company_names.keys) + '件すべてのメールの配信に成功しました。' 
  else:
    dialog[0] = '処理エラー'
    dialog[1] = '処理中にエラーが発生しました。\n' + result
  messagebox.showinfo(dialog[0], dialog[1])
  return

main()


In [ ]:
import sys
import pprint

# sys.pathのリスト表示
pprint.pprint(sys.path)